<a href="https://colab.research.google.com/github/deepmind/perception_test/blob/main/baselines/temporal_action_localisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Make sure GPU runtime is enabled!!

In [2]:
!git clone https://github.com/ptchallenge-workshop/actionformer_release_PT.git

Cloning into 'actionformer_release_PT'...
remote: Enumerating objects: 461, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 461 (delta 49), reused 54 (delta 31), pack-reused 373
Receiving objects: 100% (461/461), 664.05 KiB | 17.47 MiB/s, done.
Resolving deltas: 100% (258/258), done.


In [3]:
%cd /content/actionformer_release_PT/libs/utils
!python setup.py install --user
%cd ../..

/content/actionformer_release_PT/libs/utils
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, p

In [4]:
import os
import zipfile
import requests


def download_and_unzip(url: str, destination: str):
  """Downloads and unzips a .zip file to a destination.

  Downloads a file from the specified URL, saves it to the destination
  directory, and then extracts its contents.

  If the file is larger than 1GB, it will be downloaded in chunks,
  and the download progress will be displayed.

  Args:
    url (str): The URL of the file to download.
    destination (str): The destination directory to save the file and
      extract its contents.
  """
  if not os.path.exists(destination):
    os.makedirs(destination)

  filename = url.split('/')[-1]
  file_path = os.path.join(destination, filename)

  if os.path.exists(file_path):
    print(f'{filename} already exists. Skipping download.')
    return

  response = requests.get(url, stream=True)
  total_size = int(response.headers.get('content-length', 0))
  gb = 1024*1024*1024

  if total_size / gb > 1:
    print(f'{filename} is larger than 1GB, downloading in chunks')
    chunk_flag = True
    chunk_size = int(total_size/100)
  else:
    chunk_flag = False
    chunk_size = total_size

  with open(file_path, 'wb') as file:
    for chunk_idx, chunk in enumerate(
        response.iter_content(chunk_size=chunk_size)):
      if chunk:
        if chunk_flag:
          print(f"""{chunk_idx}% downloading
          {round((chunk_idx*chunk_size)/gb, 1)}GB
          / {round(total_size/gb, 1)}GB""")
        file.write(chunk)
  print(f"'{filename}' downloaded successfully.")

  with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(destination)
  print(f"'{filename}' extracted successfully.")

  os.remove(file_path)

In [5]:
!mkdir data
!mkdir data/pt
!mkdir ckpt

In [6]:
# @title Download data
data_path = './data/pt'
model_path = './ckpt'

train_annot_url = 'https://storage.googleapis.com/dm-perception-test/zip_data/action_localisation_train_annotations.zip'
download_and_unzip(train_annot_url, data_path)
train_feat_url = 'https://storage.googleapis.com/dm-perception-test/zip_data/action_localisation_train_video_features.zip'
download_and_unzip(train_feat_url, data_path)

valid_annot_url = 'https://storage.googleapis.com/dm-perception-test/zip_data/action_localisation_valid_annotations.zip'
download_and_unzip(valid_annot_url, data_path)
valid_feat_url = 'https://storage.googleapis.com/dm-perception-test/zip_data/action_localisation_valid_video_features.zip'
download_and_unzip(valid_feat_url, data_path)

# here we download a pretrained model, this can be commented out and the
# training command below can be ran instead to train the model from scratch
model_url = 'https://storage.googleapis.com/dm-perception-test/saved_models/perception_video_train_reproduce.zip'
download_and_unzip(model_url, model_path)

'action_localisation_train_annotations.zip' downloaded successfully.
'action_localisation_train_annotations.zip' extracted successfully.
'action_localisation_train_video_features.zip' downloaded successfully.
'action_localisation_train_video_features.zip' extracted successfully.
'action_localisation_valid_annotations.zip' downloaded successfully.
'action_localisation_valid_annotations.zip' extracted successfully.
'action_localisation_valid_video_features.zip' downloaded successfully.
'action_localisation_valid_video_features.zip' extracted successfully.
'perception_video_train_reproduce.zip' downloaded successfully.
'perception_video_train_reproduce.zip' extracted successfully.


In [7]:
# Downloading pretrained model in the cell above instead of
# training, uncomment the line below to run command for training
# !python3 train.py configs/perception_video_train.yaml --output reproduce

In [8]:
# this saves a results JSON file in this location:
# /content/actionformer_release_PT/ckpt/perception_video_train_reproduce/eval_results.json
# this is in the correct format for submission on the eval.ai challenge
# https://eval.ai/web/challenges/challenge-page/2101/overview
!python eval.py configs/perception_video_valid.yaml ckpt/perception_video_train_reproduce/

{'dataset': {'crop_ratio': [0.9, 1.0],
             'default_fps': 15,
             'downsample_rate': 1,
             'feat_folder': './data/pt/action_localisation_valid_video_features',
             'feat_stride': 16,
             'file_ext': '.npy',
             'file_prefix': 'v_',
             'force_upsampling': True,
             'input_dim': 512,
             'input_modality': 'video',
             'json_file': './data/pt/action_localisation_valid.json',
             'max_seq_len': 192,
             'mm_feat_folder': None,
             'num_classes': 63,
             'num_frames': 16,
             'trunc_thresh': 0.5},
 'dataset_name': 'perception',
 'devices': ['cuda:0'],
 'init_rand_seed': 1234567891,
 'loader': {'batch_size': 16, 'num_workers': 4},
 'model': {'backbone_arch': (2, 2, 5),
           'backbone_type': 'convTransformer',
           'embd_dim': 512,
           'embd_kernel_size': 3,
           'embd_with_ln': True,
           'fpn_dim': 512,
           'fpn_start_